<a href="https://colab.research.google.com/github/cheng1103/Test/blob/main/104_Customization_job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#104自訂條件職缺爬蟲(暫時版)

In [64]:
ID = "********" #輸入帳號
password = "********" #輸入密碼
job_op = '#li_newjob4'
file_name = '資料工程' #檔案命名


In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
#!apt-get install xvfb
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.support.ui import WebDriverWait

!pip install fake_useragent
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
from lxml import etree
import json
import pandas as pd
import random
import time

from google.colab import drive
drive.mount('/content/drive')

In [65]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [66]:
url = "https://www.104.com.tw/jobs/main/"
browser.get(url)
browser.implicitly_wait(3) #等待載入

#進入登入頁面
signin = browser.find_element_by_css_selector('#global_bk > ul > li.right > ul > li:nth-child(6) > a')
browser.execute_script("arguments[0].click();", signin)
browser.implicitly_wait(1) #等待載入

#填帳號密碼與登入
context = browser.find_element(by=By.XPATH, value='//*[@id="username"]')
context.send_keys(ID)
time.sleep(0.05) #暫停
context = browser.find_element(by=By.XPATH, value='//*[@id="password"]')
context.send_keys(password)
time.sleep(0.05) #暫停

submit = browser.find_element(by=By.XPATH, value='//*[@id="submitBtn"]/h2')
submit.click() #點擊登入
browser.implicitly_wait(3) #等待載入


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  


In [67]:
#顯示自訂工作列表
newjob = browser.find_element_by_css_selector('#bubjobs_li > a')
browser.execute_script("arguments[0].click();", newjob) #點擊
browser.implicitly_wait(3) #等待載入

#自訂工作
newjob = browser.find_element_by_css_selector(job_op)
browser.execute_script("arguments[0].click();", newjob)

#檢查並切換頁面
n = browser.window_handles  # 生成一個新視窗或新頁籤的識別碼，代表這個視窗的模擬driver
print('當前識別碼:', n)  # 列印所有頁面識別碼
browser.switch_to.window(browser.window_handles[-1])  # 瀏覽器切换至最新跳出的頁面
browser.implicitly_wait(2) #等待載入

#擷取總頁數
browser.execute_script("window.scrollTo(0,document.body.scrollHeight)") #捲動頁面至底端
time.sleep(5) #暫停
soup = BeautifulSoup(browser.page_source, 'html.parser') #解析網頁
time.sleep(5) #暫停
last_page = soup.find('div', class_="pagination__count d-inline-block").getText()
last_page = last_page.split(' ')[-3]
last_page = int(last_page) + 1

for i in range(last_page):
  browser.execute_script("window.scrollTo(0,document.body.scrollHeight)") #捲動頁面至底端
  t = random.uniform(2, 4) #隨機數
  time.sleep(t) #暫停
time.sleep(3) #暫停

#擷取職缺網址
soup = BeautifulSoup(browser.page_source, 'html.parser') #解析網頁
job_links = []
urls = soup.find_all('a', class_="info-job__text jb-link jb-link-blue jb-link-blue--visited h2")
for url in urls:
  link = url.get("href").replace('//', '').replace('?jobsource=m104_apply','')
  job_links.append(link)

browser.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  import sys


當前識別碼: ['CDwindow-7FB2B1E14A179CD74185FBCF4C282F89', 'CDwindow-3F54C062D2C0DAA4AD8BB6417C39AAF3']


In [68]:
data = pd.DataFrame()

for i in range(len(job_links)):

  #預處理網址
  link = job_links[i].replace('www.104.com.tw/job/','')
  Referer = 'https://' + job_links[i]
  url = 'https://www.104.com.tw/job/ajax/content/' + link

  #設定表頭
  useragent = UserAgent().random #隨機請求表頭
  headers = {'User-Agent':useragent, 'Referer':Referer }

  #發送請求
  req = requests.get(url, headers=headers)
  t = random.uniform(0.1, 1.5) #隨機數
  time.sleep(t) #暫停
  req = req.json()['data']

  #擷取資料
  date = req['header']['appearDate'] #刊登日期
  company = req['header']['custName'] #公司
  jobName = req['header']['jobName'] #職銜
  jobDescription = req['jobDetail']['jobDescription']#.replace('\n', '　') #工作內容
  jobcondition = req['condition']['other']#.replace('\n', '　') #工作條件
  businessTrip = req['jobDetail']['businessTrip'] #外派
  address = req['jobDetail']['addressRegion'] + req['jobDetail']['addressDetail'] + ' | ' + req['jobDetail']['landmark'] #地址

  expertise = req['condition']['specialty'] #專長-------------
  speciality = ''

  if len(expertise) == 0 :
    speciality = '不限'

  else:
    for i in range(len(expertise)):
      temp = expertise[i]['description']
      speciality = speciality + ' |' + temp

  certificate = req['condition']['certificate'] #證照
  license = ''

  if len(certificate) == 0 :
    license = '未要求'

  else:
    for i in range(len(certificate)):
      temp = certificate[i]['name']
      license = license + ' |' + temp


  #彙整資料
  temp_data = {(date , company, jobName, jobDescription, jobcondition, speciality, license, businessTrip, address, Referer)}
  temp_data = pd.DataFrame(temp_data)
  data = data.append(temp_data)


In [69]:
data.columns = [ '日期', '公司', '職銜', '工作內容', '其他條件', '擅長工具', '證照', '出差與外派', '地區與地址', '網址']
data

,日期,公司,職銜,工作內容,其他條件,擅長工具,證照,出差與外派,地區與地址,網址
0,2022/04/26,恆生金融科技股份有限公司,前端工程師,恆生金融為「金融科技貸款規劃顧問公司」，結合金融科技(FinTech)、AI大數據統計，協助...,,|Github |AJAX |ASP.NET |JSF |MS SQL |MySQL |H...,未要求,無需出差外派,新北市板橋區板新路101號8樓（板新捷運站旁） | 距捷運板新站60公尺,https://www.104.com.tw/job/6u7fz
0,2022/04/26,元大金融控股股份有限公司(元大金控) (元大證券) (元大銀行),元大證券-《研發替代役》金融科技研發人員,,1.具備程式邏輯概念，熟悉RWD、APP開發、API應用、Python、VBA、MS SQL...,|Linux |C |C# |C++ |Matlab |Python |VBA |PL/S...,未要求,無需出差外派,台北市中山區南京東路二段77號8樓 | 距捷運松江南京站190公尺,https://www.104.com.tw/job/5u9qy
0,2022/04/26,台中商業銀行股份有限公司,資訊人員-程式設計-(台北地區),1.大學以上畢業，曾修習資訊相關課程者。\r\n2.熟悉COBOL、VBA、VB、ASP.N...,1.大學以上畢業，曾修習資訊相關課程者。\r\n2.熟悉COBOL、VBA、VB、ASP.N...,不限,未要求,無需出差外派,台北市中山區 |,https://www.104.com.tw/job/6cwzr
0,2022/04/26,O-Bank_王道商業銀行股份有限公司,資料庫管理人員,1.SQL Server 安裝、管理、維護。\n2.Oracle 及SQL 程式過版。\n3...,1.有使用及管理MS SQL Server 經驗。\n2.學歷要求:大學以上\n3.科系要求...,不限,未要求,無需出差外派,台北市內湖區 |,https://www.104.com.tw/job/5975h
0,2022/04/26,德信綜合證券股份有限公司,資訊程式設計與系統維護,1. 主辦現有業務應用系統維運\n2. 協助辦理新業務應用程式設計開發\n3.協助辦理網頁設...,1.反應敏捷\r\n2.具創意 \r\n3.責任心,|ASP.NET |C# |MS SQL,未要求,無需出差外派,台北市中正區新生南路一段50號3樓 | 距捷運忠孝新生站30公尺,https://www.104.com.tw/job/7aw2t
...,...,...,...,...,...,...,...,...,...,...
0,2021/12/13,富邦綜合證券股份有限公司,富邦證券-ETF及量化策略交易員,(1)協助進行量化交易及ETF避險策略研究。\n(2)協助進行交易模型回測及程式開發。,(1)熟悉統計模型且具備數據分析經驗者佳\n(2)熟悉VBA、PYTHON語法。\n(3)具...,不限,|證券商高級業務員 |期貨商業務員,無需出差外派,台北市大安區仁愛路四段169號 | 距捷運忠孝敦化站400公尺,https://www.104.com.tw/job/7ay2a
0,2021/11/04,星展(台灣)商業銀行股份有限公司,(約聘六個月) CBG Credit_ Data Steward/Partner,o Be the backup/support for CBG Credit BAU ope...,o Bachelor degree / University degree\no Good ...,不限,未要求,無需出差外派,台北市內湖區 |,https://www.104.com.tw/job/7fbvd
0,2021/10/25,星展(台灣)商業銀行股份有限公司,Software Engineer Team Lead,1. Develop technology applications utilizing t...,1. Seasoned coders with people management expe...,不限,未要求,無需出差外派,台北市內湖區 |,https://www.104.com.tw/job/7b6iw
0,2021/10/25,星展(台灣)商業銀行股份有限公司,Finance COO Assistance(6 months),1. Support Project manager to control project ...,•\tPrimary accounting knowledge is must. Oper...,不限,未要求,無需出差外派,台北市內湖區 |,https://www.104.com.tw/job/7avk5


In [70]:
#輸出檔案
path = '/content/drive/MyDrive/Data Set/'+file_name + '.csv' #路徑
data.to_csv(path, index=None)